In [ ]:
from os import environ, makedirs
from os.path import join
subjects_dir = '/Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir'
subject = '030_WAH'
lab_path = join(subjects_dir, subject, 'label')
environ['SUBJECTS_DIR'] = subjects_dir
environ['SUBJECT'] = subject

# Retinotopic target locations

This notebook generates the following Freesurfer annotation-files. Each file contains the cortical patch information for the retinotopic target locations defined earlier:

- `[l|r]h.RM.annot`
    - all labels, regardless of region
- `[l|r]h.RM.V[1|2|3].annot`
    - labels restricted to a particular visual region
    
## Note on colormap & LUT

For some inexplicable reason, Freesurfer annotations do not work if labels share a colour: each label gets the name of the first region found in the LUT with the same colour! To avoid this, each region __must__ have a unique RGB colour.

So, we'll paint the locations with eccentricity-dependent decline of saturation!

### Must define None

If not, `freeview` crashes on loading the `.annot`-file...?

In [ ]:
%matplotlib inline

In [ ]:
from retinotopic_helpers import *
import subprocess as subp
import glob
import json
import matplotlib.pyplot as plt

In [ ]:
# r, g, b = 0., 1.0, 1.0
# h, s, v = 245/360, 1., 1.
# print(colorsys.rgb_to_hsv(r, g, b))
# print(hsv2rgb_int(h, s, v))
# print(colorsys.hsv_to_rgb(h,s,v))

In [ ]:
th_plot = []  # these are the starting angles, NOT centers
r_plot = []
w_plot = []
b_plot = []
c_plot = []
for ii, th_start in enumerate(theta_starts):
    wedge = [th_start, 2*np.pi - (th_start + theta_width)]
    color = cm.jet((np.abs(th_start) + theta_width/2)/np.pi)
    h, s, v = colorsys.rgb_to_hsv(*color[:-1])
    for jj, rlim in enumerate(radii):
        th_plot.extend(wedge)
        w_plot.extend([theta_width, theta_width])
        r_plot.extend([np.diff(rlim)[0], np.diff(rlim)[0]])
        b_plot.extend([rlim[0], rlim[0]])
        
        v = 1 - (jj / len(radii)) / 2
        col = colorsys.hsv_to_rgb(h,s,v)
        c_plot.extend([col, col])    
#          ind_plot.append("A{:d}_E{:d}".format(ii, jj))

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111, projection='polar')
ax.set_theta_zero_location("N")
bars = ax.bar(th_plot, r_plot, width=w_plot, bottom=b_plot)
for c, bar in zip(c_plot, bars):
    bar.set_facecolor(c)
#     bar.set_alpha(1. - (r - 1.2)/4.)

Binary encoding of labels requires 9 bits. Let the first 6 bits determine the angle, the following 3 the eccentricity. __We do not need to encode the hemisphere, as each hemisphere has its own angle definition (counter-clockwise on left, clockwise on right).__

__NB!__ Instead of "proper" bits, counting right-to-left, we will just assign left-to-right. The numbers will not be "logical", but the code will!

In [ ]:
#         print("{:d}: {:s} -> {:d}".format(index + 1, bin_code,
#                                           int(bin_code, 2)))


To find the up bits, use this later:
```python
import re
[m.start() for m in re.finditer('1', bin(1153)[2:])]
```

In [ ]:
if not isfile('RMLUT.txt'):
    create_LUT()

In [ ]:
bin_cmd = 'mri_binarize'
bin_args = ("--i {path:s}/{hemi:s}.benson.{map:s}.mgh "
            "--min {min:.1f} --max {max:.1f} "
            "--o {path:s}/tmp/{hemi:s}.{map:s}.tmp.nii")
calc_cmd = 'fscalc'
calc_args = ("{path:s}/tmp/{hemi:s}.{emap:s}.tmp.nii mul "
             "{path:s}/tmp/{hemi:s}.{amap:s}.tmp.nii mul {mult:d} "
             "--o {path:s}/tmp/{hemi:s}.RM{code:d}.tmp.nii")

In [ ]:
VERBOSE = True
FAKE = False
for ii_code, bin_code in enumerate(code_map):
    ups = [m.start() for m in re.finditer('1', bin_code)]

    ecc_ind = ups[0]
    ang_ind = ups[1] - len(radii)

    for hemi in ['lh', 'rh']:
        cur_args = bin_args.format(hemi=hemi, min=radii[ecc_ind][0],
                                   max=radii[ecc_ind][1], map='eccen',
                                   path=lab_path)
        execute(bin_cmd + ' ' + cur_args, verbose=VERBOSE, fake=FAKE)

        ang_min, ang_max = (np.abs(theta_starts_deg[ang_ind]),
                            np.abs(theta_starts_deg[ang_ind]) + wedge_width)
        cur_args = bin_args.format(hemi=hemi, min=ang_min,
                                   max=ang_max, map='angle', path=lab_path)

        execute(bin_cmd + ' ' + cur_args, verbose=VERBOSE, fake=FAKE)


        cur_args = calc_args.format(hemi=hemi,
                                    emap='eccen', amap='angle',
                                    mult=ii_code + 1,
                                    code=code_map[bin_code],
                                    path=lab_path)
        execute(calc_cmd + ' ' + cur_args, verbose=VERBOSE, fake=FAKE)


In [ ]:
VERBOSE = True
FAKE = False
masks = dict()
masks['lh'] = glob.glob(join(subjects_dir, subject, 'label/tmp/lh.RM*'))
masks['rh'] = glob.glob(join(subjects_dir, subject, 'label/tmp/rh.RM*'))
makedirs(join(lab_path, 'RM'), exist_ok=True)

for hemi in masks.keys():
    calc_cmd = ['fscalc']
    for mask_fname in masks[hemi]:
        calc_cmd += [mask_fname, 'add']
        
    RM_out = join(subjects_dir, subject, 'label/{:s}.RM.nii'.format(hemi))
    calc_cmd = (' '.join(calc_cmd[:-1]) + ' --o ' + RM_out)
    
    execute(calc_cmd, verbose=VERBOSE, fake=FAKE)

    cmd = ('mris_seg2annot --seg {path:s}/{hemi:s}.RM.nii --ctab RMLUT.txt '
           '--s {subject:s} --h {hemi:s} '
           '--o {path:s}/{hemi:s}.RM.annot').format(path=lab_path, hemi=hemi,
                                                    subject=subject)
    execute(cmd, verbose=VERBOSE, fake=FAKE)

    
    # Make separate annots for each of V1, V2 and V3
    for regno in range(1, 4):
        cur_args = bin_args.format(hemi=hemi, min=regno - 0.1,
                                   max=regno + 0.1, map='areas',
                                   path=lab_path)
        cur_args = '--abs ' + cur_args

        execute(bin_cmd + ' ' + cur_args, verbose=VERBOSE, fake=FAKE)

        areas_out = '{path:s}/tmp/{hemi:s}.areas.tmp.nii'.format(path=lab_path,
                                                                 hemi=hemi)
        cmd = ('fscalc {rm:s} mul {area:s} '
               '--o {path:s}/{hemi:s}.RM.V{regno:d}.nii'
               '').format(rm=RM_out, area=areas_out,
                             path=lab_path, hemi=hemi, regno=regno)

        execute(cmd, verbose=VERBOSE, fake=FAKE)


        cmd = ('mris_seg2annot --seg {path:s}/{hemi:s}.RM.V{regno:d}.nii '
               '--ctab RMLUT.txt --s {subject:s} --h {hemi:s} --o '
               '{path:s}/{hemi:s}.RM.V{regno:d}.annot'
               '').format(path=lab_path, hemi=hemi,
                          subject=subject, regno=regno)
        execute(cmd, verbose=VERBOSE, fake=FAKE)


## Calculate patch areas

In [ ]:
cmd = 'mris_anatomical_stats'
args = (" -a {path:s}/{hemi:s}.RM.{reg:s}.annot "
        "{subj:s} {hemi:s} > tmp.out")

In [ ]:
VERBOSE = True
FAKE = False
areas = dict()
for reg in ['V1', 'V2', 'V3']:
    areas[reg] = dict()
    for hemi in ['lh', 'rh']:
        areas[reg][hemi] = dict()
        cur_args = args.format(path=lab_path, hemi=hemi, reg=reg, subj=subject)
        
        execute(cmd + cur_args, verbose=VERBOSE, fake=FAKE)
        
        with open('tmp.out', 'rt') as fp:
            for line in fp:
                if "structure is" in line:
                    lab = line.split("\"")[1]
                if "total surface area" in line:
                    area = line.split('=')[1].split(' ')[1]  # mm^2
                    areas[reg][hemi][lab] = area

In [ ]:
# for posterity
with open('area_stats.json', 'wt') as fp:
    json.dump(areas, fp)